# Run multiple rainfall events
This example shows how to run multiple rainfall events. The basic workflow is:
1. Explore the rainfall boundary conditions of a dfs0 file using MIKE IO.
2. Create a boundary condition for each rainfall event. Set them all as inactive by default.
3. Run the model for each event by activating the relevant boundary condition and updating the scenario name.

#### Start with some imports we'll need

In [ ]:
import clr
import System
import mikeio
from mikeplus import DataTableAccess
from mikeplus.engines import Engine1D
from pathlib import Path
from mikeplus.fieldTableNames.tableNames import CSTabNames
from mikeplus.fieldTableNames.fieldNames import Fields

#### 1. Explore rainfall boundary dfs0 file

In [ ]:
# Let's start by opening the dfs0 file that contains our rainfall events.
# This file contains nine events with varying frequencies.
# Our goal is to run the model for every single one of these events.
rainfall_timeseries = Path("../tests/testdata/Db/Sirius/Sirius_IDF_rainfall.dfs0")
ds = mikeio.read(rainfall_timeseries)
ds

#### 2. Create boundary conditions for each rainfall event

In [ ]:
# First we open the MIKE+ database using MIKE+Py.
db_path = Path("../tests/testdata//Db/Sirius/Sirius.sqlite")
dta = DataTableAccess(db_path)
dta.open_database()
dta.is_database_open()

In [ ]:
# We'll need the dfs0 file path, the item name, and data type. 
# This is the same info required to add boundary conditions in MIKE+.
event_info = []
for item in ds.items:
    event_info.append(
        {
                         # The dfs0 path should be relative to the .sqlite path
            "file_path": str(rainfall_timeseries.relative_to(db_path.parent)), 
            "item_name": item.name,
            "data_type": repr(item.type),
        }
    )

# Peak at the first two event infos created
event_info

In [ ]:
# Now let's create a boundary condition for each event using this info.
boundary_muids = []
for event in event_info:
    values = dict(
        VariationNo = System.Nullable[int](3), # set temporal variation to timeseries
        FlowTypeNo = System.Nullable[int](5), # set boundary condition type to Rainfall
        LoadTypeNo = System.Nullable[int](1), # set load type to stormwater runoff
        TSConnection = event['file_path'],
        TimeseriesName = event['item_name'],
        DataTypeName = event['data_type'],
    )
    muid = "Rainfall " + event['item_name']
    dta.insert(CSTabNames.BBoundaryTable, muid, values)
    dta.set_value(CSTabNames.BBoundaryTable, muid, Fields.BBoundaryFields.ApplyBoundaryNo, System.Nullable[int](0)) # Set boundary as inactive
    boundary_muids.append(muid)

# Peek at the first 3 boundary MUIDs created
# Hint: check out if they've been created in MIKE+ by clicking the 'Clear' button in the 'Boundary conditions' table.
boundary_muids

#### 3. Run the model for each event

In [ ]:
# Create the Engine1D object that will be used to run MIKE 1D simulations.
engine = Engine1D(dta.datatables)

In [ ]:
# Confirm which simulation is marked as the active one
active_simulation = dta.get_muid_where(CSTabNames.ProjectTable, f"{Fields.ProjectFields.ActiveProject} == 1")[0]
active_simulation

In [ ]:
# Create a little helper function to rename the project simulation
def rename_simulation(current_muid, new_muid):
    # Rename the simulation
    dta.set_value(CSTabNames.ProjectTable, current_muid, Fields.FieldNameBase.MUID, new_muid)

    # Change simulation MUID in the msm_ProjectOutput table too (will be fixed in future to happen automatically)
    output_muids = dta.get_muid_where(CSTabNames.ProjectOutputTable, f"{Fields.ProjectOutputFields.SimulationID} == '{current_muid}'")
    for muid in output_muids:
        dta.set_value(CSTabNames.ProjectOutputTable, muid, current_muid, new_muid)

In [ ]:
# Run the simulations
current_simulation_muid = active_simulation
for boundary_muid in boundary_muids:
    dta.set_value(CSTabNames.BBoundaryTable, boundary_muid, Fields.BBoundaryFields.ApplyBoundaryNo, System.Nullable[int](1)) # Set boundary as active
    new_simulation_muid = "Simulation " + boundary_muid
    rename_simulation(current_simulation_muid, new_simulation_muid)
    engine.run(new_simulation_muid)
    current_simulation_muid = new_simulation_muid
    dta.set_value(CSTabNames.BBoundaryTable, boundary_muid, Fields.BBoundaryFields.ApplyBoundaryNo, System.Nullable[int](0)) # Set boundary as inactive again

In [ ]:
# Check that the result files for each simulation have been created
results = [p for p in db_path.parent.glob(f"*Network_HD.res1d")]
for r in results:
    print(r)